In [1]:
from transformers import TrainingArguments, Trainer, AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification
from datasets import load_dataset
from scipy.special import softmax
import evaluate
import numpy as np

In [2]:
ds = load_dataset('imdb')

In [3]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [4]:
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

In [6]:
def preprocess_text(ds):
    return tokenizer(ds["text"], truncation=True)

In [7]:
tokenized_ds = ds.map(preprocess_text, batched=True)

In [8]:
roc_auc = evaluate.load('roc_auc', 'binary')

In [9]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    pred = np.argmax(logits, axis=1)
    probabilities = softmax(pred)
    return roc_auc.compute(prediction_scores=probabilities, references=labels)

In [10]:
small_train = tokenized_ds["train"].shuffle(seed=42).select(range(100))
small_eval = tokenized_ds["test"].shuffle(seed=42).select(range(100))

In [11]:
training_args = TrainingArguments(
    output_dir="sentiment",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds['train'],
    eval_dataset=tokenized_ds['test'],
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

In [13]:
trainer.train()

Epoch,Training Loss,Validation Loss,Roc Auc
1,0.224300,0.198599,0.923680
2,0.143400,0.240821,0.931440


TrainOutput(global_step=3126, training_loss=0.20339089879910305, metrics={'train_runtime': 753.824, 'train_samples_per_second': 66.328, 'train_steps_per_second': 4.147, 'total_flos': 6556904415524352.0, 'train_loss': 0.20339089879910305, 'epoch': 2.0})

In [35]:
# Using SKLearn ROC-AUC

from sklearn.metrics import roc_auc_score
dic={}
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    pred = np.argmax(logits, axis=1)
    probabilities = softmax(pred)
    dic['roc_auc'] = roc_auc_score(y_true=labels, y_score=probabilities)
    return dic

In [38]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds['train'],
    eval_dataset=tokenized_ds['test'],
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

In [39]:
trainer.train()

Epoch,Training Loss,Validation Loss,Roc Auc
1,0.193100,0.052911,0.981132
2,0.109000,0.045104,0.981132


TrainOutput(global_step=3126, training_loss=0.16125304624199943, metrics={'train_runtime': 577.0516, 'train_samples_per_second': 86.647, 'train_steps_per_second': 5.417, 'total_flos': 6542215022895264.0, 'train_loss': 0.16125304624199943, 'epoch': 2.0})

In [42]:
dic

{'roc_auc': 0.9811320754716981}